## 4 Simulation experiments on the impact of $L$ and strategies
### 4.2 Strategies study

In [1]:
import os
path = os.getcwd() + '/Results/'

os.chdir('../')
from inference_mahmm import *

import seaborn as sns
from tqdm.notebook import tqdm

palette = sns.color_palette("husl", 5)

##### Results recuperation

In [2]:
M = np.array([-.04, 0, .02])
mindiff_M = np.min(np.abs(M[0]-M[1:])), np.min(np.abs(M[1]-M[[0,2]])), np.min(np.abs(M[2]-M[:-1]))

Σ, seeds, L = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1], np.arange(1000), [0, 2, 5, 8, 10]
fees = [0, 0.0005, 0.001, 0.002]

In [3]:
ir, ir_c = np.zeros((len(Σ), len(seeds), len(L), len(fees))), np.zeros((len(Σ), len(seeds), len(L), len(fees)))

for si, σ in enumerate(Σ):
    for seedi, seed in tqdm(enumerate(seeds)):
        
        y = np.asarray(np.load(path + 'y/y_sigma' + str(σ) + '_' + str(seed) + '.npy', 'r'))
        true_x = np.asarray(np.load(path + 'true_x/true_x_' + str(seed) + '.npy', 'r'))
        
        for li, l in enumerate(L):
            with open(path + 'res/res_up_L' + str(l) + '_sigma' + str(σ) + '_' + str(seed) + '.pickle', 'rb') as handle:
                res = pickle.load(handle)
                
            x_c = res['x_c']
            x = res['x_b']

            if res['M'][1] > 0:
                x[x == 1] = 2
            else:
                x[x == 1] = 0
        
            ir[si, seedi, li] = [compute_sr_strat(x, y[-len(x):]/100, np.zeros(len(x)), fees = fee) for fee in fees]
            ir_c[si, seedi, li] = [compute_sr_strat(x_c, y[-len(x):]/100, np.zeros(len(x)), fees = fee) for fee in fees]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

##### Binary signal

In [4]:
for fi, fee in enumerate(fees):
    df_melted = pd.DataFrame([[l, σ, ir[si, seedi, li][fi]] for si, σ in enumerate(Σ) for seedi, seed in enumerate(seeds) for li, l in enumerate(L)])
    df_melted.columns = ['L', 'Sigma', 'Value']
    df_melted['ASDS'] = [np.round(np.mean(np.array(s)/mindiff_M),2) for s in df_melted['Sigma']]
    
    df_melted = df_melted.groupby(['ASDS', 'L']).median()['Value'].round(2)
    
    if fi>0: print(f'\n{fee*1000}bp transaction fees')
    else: print('No transaction fee')
    
    print(pd.DataFrame(df_melted).pivot_table(index='ASDS', columns='L', values='Value'))

No transaction fee
L        0      2      5      8      10
ASDS                                   
0.42  17.18  14.72  13.21  11.78  11.00
0.83  10.99  10.19   9.33   8.55   8.14
1.67   5.22   5.13   4.88   4.56   4.43
2.50   3.15   3.20   3.06   2.88   2.83
3.33   2.28   2.27   2.17   2.11   2.07
4.17   1.69   1.75   1.66   1.63   1.59

0.5bp transaction fees
L       0     2     5     8     10
ASDS                              
0.42  9.34  7.14  8.06  7.94  7.64
0.83  6.81  3.13  5.07  5.46  5.47
1.67  3.31 -0.32  1.72  2.26  2.45
2.50  0.75 -1.08  0.55  1.12  1.30
3.33 -0.29 -1.23  0.14  0.63  0.78
4.17 -0.95 -1.17 -0.01  0.39  0.52

1.0bp transaction fees
L       0     2     5     8     10
ASDS                              
0.42  4.27  2.40  3.95  4.38  4.40
0.83  3.40 -0.87  1.76  2.64  2.95
1.67  1.71 -3.65 -0.71  0.32  0.64
2.50 -0.95 -4.07 -1.49 -0.49 -0.09
3.33 -2.43 -3.93 -1.57 -0.69 -0.38
4.17 -2.97 -3.55 -1.53 -0.77 -0.49

2.0bp transaction fees
L       0     2     5     8  

##### Continuous signal

In [5]:
for fi, fee in enumerate(fees):
    df_melted = pd.DataFrame([[l, σ, ir_c[si, seedi, li][fi]] for si, σ in enumerate(Σ) for seedi, seed in enumerate(seeds) for li, l in enumerate(L)])
    df_melted.columns = ['L', 'Sigma', 'Value']
    df_melted['ASDS'] = [np.round(np.mean(np.array(s)/mindiff_M),2) for s in df_melted['Sigma']]
    
    df_melted = df_melted.groupby(['ASDS', 'L']).median()['Value'].round(2)
    
    if fi>0: print(f'\n{fee*1000}bp transaction fees')
    else: print('No transaction fee')
    
    print(pd.DataFrame(df_melted).pivot_table(index='ASDS', columns='L', values='Value'))

No transaction fee
L        0      2      5      8      10
ASDS                                   
0.42  17.29  15.58  13.36  12.09  11.36
0.83  10.61  10.33   9.39   8.78   8.51
1.67   5.17   5.39   5.16   4.94   4.85
2.50   3.38   3.47   3.39   3.25   3.22
3.33   2.49   2.53   2.48   2.41   2.35
4.17   1.96   2.00   1.97   1.91   1.88

0.5bp transaction fees
L       0     2     5     8     10
ASDS                              
0.42  9.52  7.88  9.04  8.91  8.64
0.83  5.96  4.35  6.13  6.41  6.46
1.67  2.32  0.89  2.86  3.35  3.46
2.50  0.55 -0.04  1.63  2.02  2.15
3.33  0.10 -0.30  1.09  1.41  1.50
4.17 -0.03 -0.35  0.79  1.05  1.13

1.0bp transaction fees
L       0     2     5     8     10
ASDS                              
0.42  4.02  2.38  5.25  5.93  6.09
0.83  1.96 -0.62  3.05  4.08  4.45
1.67 -0.18 -3.10  0.69  1.79  2.07
2.50 -1.87 -3.22 -0.11  0.81  1.10
3.33 -2.12 -2.90 -0.31  0.43  0.63
4.17 -1.94 -2.56 -0.38  0.22  0.43

2.0bp transaction fees
L       0     2     5     8  